In [2]:
from langchain_community.llms import Tongyi

In [3]:
MGZD_DICT={
    '运维':'操作'
    ,'运营':'操作'
    ,'扩容':'操作'
    ,'迁移':'操作'
    ,'国产化':'适配'
    ,'重构':'构建'
    ,'脚本':'代码语句'
    ,'开发':'设计语言'
    ,'环境搭建':'配置基础环境'
    ,'巡检':'代码检查'
    ,'监控':'输出运行状态'
    ,'收入保障':'读取'
    ,'工单':'任务清单'
    ,'新集群功能':'系统功能'
    ,'告警':'异常信息通知'
    ,'数据迁移':'读取数据，并输出到指定环境'
    ,'数据维护':'SQL代码生成'
    ,'分析':'检测'
    ,'验证':'鉴定'
    ,'调度':'鉴权'
    ,'采集':'提取'
    ,'整理':'转化'
    ,'处理':'响应'
    ,'配置':'设置基础属性'
    ,'测试':'鉴定'
}
print(MGZD_DICT.keys())

dict_keys(['运维', '运营', '扩容', '迁移', '国产化', '重构', '脚本', '开发', '环境搭建', '巡检', '监控', '收入保障', '工单', '新集群功能', '告警', '数据迁移', '数据维护', '分析', '验证', '调度', '采集', '整理', '处理', '配置', '测试'])


In [4]:
#　通义模型选择　https://help.aliyun.com/zh/dashscope/developer-reference/tongyi-thousand-questions-metering-and-billing?spm=a2c4g.11186623.0.0.5873fa70fW9mby
# 对话模型 https://help.aliyun.com/zh/dashscope/developer-reference/tongyi-qianwen-7b-14b-72b-metering-and-billing?spm=a2c4g.11186623.0.0.37b03a477kSxQb
#　初始化llm
from langchain_community.llms import Tongyi
llm = Tongyi(
    model="qwen1.5-110b-chat",
    temperature=0,
    max_tokens=1024,
    streaming=True,
)


In [5]:
#　测试一下模型是不是好用
llm.invoke("""你是写COSMIC的专家，帮我写COSMIC,功能过程是:"客户收入统计",根据我给定的功能过程，拆分出四个子过程描述，子过程需要包含谓语，并生成对应的数据组和数据属性，要求数据属性有3个，且不可重复，数据组文字不可重复。并以如下格式输出:子过程描述|数据组|数据属性
""")

'子过程1: 获取客户信息|客户数据库|客户ID, 收入来源, 交易日期\n- 描述：此子过程从客户数据库中检索客户的详细信息，特别是与收入相关的数据，包括客户的唯一识别ID、收入的来源（如工资、投资等）以及交易发生的日期。\n\n子过程2: 录入交易详情|交易记录表|交易ID, 客户ID, 金额\n- 描述：在交易记录表中记录每次交易的详细信息，包括为每笔交易分配一个唯一的交易ID、关联的客户ID以及交易的金额。\n\n子过程3: 分类汇总收入|收入分类报表|分类标签, 总金额, 统计周期\n- 描述：这一过程涉及将收入按照不同的类别（如月度、季度或年度）进行分类汇总，生成报表，其中包括分类的标签（时间周期或收入类型），每个分类的总金额，以及统计的时间周期。\n\n子过程4: 校验与审核收入数据|数据验证日志|验证状态, 审核日期, 差异说明\n- 描述：对统计的收入数据进行校验和审核，记录数据验证日志，包含验证的状态（通过或未通过）、进行审核的日期，以及任何发现的数据差异的具体说明。\n\n这些子过程涵盖了从获取原始客户和交易信息，到处理和验证这些信息，最终形成统计分析的完整流程，确保了客户收入统计的准确性和完整性。'

In [12]:
# 测试langchin 模板功能
from langchain.prompts import PromptTemplate
template="""
    你是写COSMIC的专家，帮我写COSMIC,功能过程是:"{text}",根据我给定的功能过程，拆分出四个子过程描述，子过程需要包含谓语，并生成对应的数据组和数据属性，要求数据属性有3个，且不可重复，数据组文字不可重复。并以如下格式输出:子过程描述|数据组|数据属性
"""
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)
chanin = prompt | llm

ret =chanin.invoke({"text":"客户收入统计"})
print(ret)

子过程1: "收入数据收集"
| 数据组1: 客户交易记录 | 数据属性1: 交易ID | 数据属性2: 交易金额 | 数据属性3: 交易日期 |

子过程2: "数据清洗与验证"
| 数据组2: 清洗后交易数据 | 数据属性1: 验证状态 | 数据属性2: 客户ID | 数据属性3: 有效金额 |

子过程3: "按客户分类"
| 数据组3: 客户分类统计 | 数据属性1: 客户类别 | 数据属性2: 类别总收入 | 数据属性3: 客户数量 |

子过程4: "收入总计与分析"
| 数据组4: 收入汇总报告 | 数据属性1: 总收入 | 数据属性2: 平均收入 | 数据属性3: 收入趋势(季度/年度) |


In [6]:
# 帮写COSMIC的prompt
from langchain.prompts import ChatPromptTemplate
from langchain.chains.sequential import SimpleSequentialChain, SequentialChain
# 顺序链 SimpleSequentialChain & SequentialChain
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatTongyi
chat_llm = ChatTongyi(
    temperature=0,
    max_tokens=1024,
    model='qwen1.5-110b-chat',
    # model="qwen-max",
    # model ='qwen-plus',
    streaming=True,
    verbose=True,
)

# 你是写COSMIC的专家，帮我写COSMIC,功能过程是:"{xuqiu_name}",根据我给定的功能过程，拆分出四个子过程描述，子过程需要包含谓语(只能是：输入、查询、展示、写入)，并生成对应的数据组和数据属性，要求数据属性有3个，且不可重复，数据组文字不可重复。并以如下格式输出:子过程描述|数据组|数据属性
template0="""
你是一个电信行业运营商的产品经理，根据我给定的需求名称、需求描述将需求拆分成{n}个功能点,要求功能点字数在50~80个字之间,功能点之间以逗号作为分隔符号，不要有其他多余的字符。
需求名称：{xqmc}
功能点不能超出如下范围:
{xqms}
"""
prompt0 = ChatPromptTemplate.from_template(template0)
template0_1 ="""
你是一个电信行业运营商的产品经理，总结并润色一下如下需求描述，要求字数1000个字以上。
需求名称如下:{xqmc}
报表指标如下:{xqms}
"""
prompt0_1 = ChatPromptTemplate.from_template(template0_1)

# 拆分子功能过程
template1 = """
你是一个电信行业运营商的产品经理，现在需要根据给定的功能过程:"{xuqiu_name}"完成COSMIC子功能过程的拆分，要求如下:
1.需要拆分成4个子功能过程，子功能过程字数在30~50个字之间
2.子功能过程必须包含谓语，且谓语只能是：输入、查询、展示、写入
3.谓语不能重复
4.子过程中不可包含如下字眼：'运维', '运营', '扩容', '迁移', '国产化', '重构', '脚本', '开发', '环境搭建', '巡检', '监控', '收入保障', '工单', '新集群功能', '告警', '数据迁移', '数据维护', '分析', '验证', '调度', '采集', '整理', '处理', '配置', '测试','界面','配置','汇总'
5.根据子过程生成对应的数据组和数据属性，其中数据组不能重复，数据属性至少要有3个，且不能重复
6.子过程不可包含如下子过程:"{his_zgc}"
"""


template2 ="""
从我给定的文本中提取:子过程、谓语、数据组、数据属性,以如下格式输出"子过程描述|谓语|数据组|数据属性\n"，文本内容如下\n{cosmic_str}
"""

template3="""
你是一个严格的文本检测人员，你只能输出:"正确"和"错误"两种结果，你需要基于如下规则对我输入的文本进行检测：
1.文本中如果包含如下字眼：'运维', '运营', '扩容', '迁移', '国产化', '重构', '脚本', '开发', '环境搭建', '巡检', '监控', '收入保障', '工单', '新集群功能', '告警', '数据迁移', '数据维护', '分析', '验证', '调度', '采集', '整理', '处理', '配置', '测试','界面','配置','汇总' 输出"错误"
2.如果内容格式类似："子功能过程1|输入|标准规范库|标准代码, 标准名称, 适用范围" ，这种没有功能过程详细的描述输出"错误"
3.如果内容格式类似："1. 子功能过程1: 输入数据规范设定|输入|数据输入规范|数据格式, 强制性字段, 数据验证规则" 输出"正确"
4.如果内容格式类似："1. 输入物联网设备信息|输入|设备信息库|设备ID, 设备类型, 网络协议" 输出"正确"
5.如果只有子功能过程，没有":"描述，输出"错误"
文本内容如下：
{content}
如果有多条返回结果，则合并结果，全部"正确"，返回"正确"，只要有一个"错误"，返回"错误"
"""

template4 ="""
提取出如下文本中每行第一个'|'之前的文字，逗号分隔，不显示多余字段：
{value}
"""

prompt1=ChatPromptTemplate.from_template(template1)
prompt2 = ChatPromptTemplate.from_template(template2)
prompt3 = ChatPromptTemplate.from_template(template3)
prompt4 = ChatPromptTemplate.from_template(template4)
chanin_1 = LLMChain(
    llm = chat_llm
    ,prompt = prompt1
    ,verbose = True
    ,output_key = "cosmic_str"
)
chain_2 = LLMChain(
    llm = chat_llm
    ,prompt = prompt2
    ,verbose = True
    ,output_key = "k2"
)



simp_sq_chain2 = SequentialChain(
    chains=[chanin_1, chain_2],
    verbose=True,
    input_variables=["xuqiu_name","his_zgc"],
    output_variables=["k2"],
)



C:\Users\86158\AppData\Local\Temp\ipykernel_444\4051372509.py:69: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chanin_1 = LLMChain(


In [7]:
# 获取数据
import time
from langchain.schema import BaseOutputParser

MGC =['运维', '运营', '扩容', '迁移', '国产化', '重构', '脚本', '开发', '环境搭建', '巡检', '监控', '收入保障', '工单', '新集群功能', '告警', '数据迁移', '数据维护', '分析', '验证', '调度', '采集', '整理', '处理', '配置', '测试','汇总']

def check2(str,my_list):

    for i in my_list:
        if i in str:
            return False
    return True

chanin = prompt0 | chat_llm
zongjiechain = prompt0_1|chat_llm
jcy_chain = prompt3|chat_llm
zgc_tq = prompt4|chat_llm
# 检查合规性
def check_list(str,my_list):
    if '子功能过程1|' in str:
        return '错误'
    if '子功能过程一|' in str:
        return '错误'
    for i in my_list:
        if i in str:
            return '错误'
    return '正确'

try_max =1
def gent_zgc(str):
    ret = ''
    for line in str.split('\n'):
        if '|' in line:
            line = line.split('|')[0]
            line = line.split(':')[-1] if ':' in line else line
            line = line.split('：')[-1] if '：' in line else line
            ret += line + ','
    return ret

def go(xqmc,xqms,n,profex):
    with open('E:\\输出.txt','a') as f:
        f.write('\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n')
        f.write(f'需求名称:{xqmc}\n')
    xuqiumingcheng =xqmc
    qiefengeshu = n
    class CommaSeparatedListOutputParser(BaseOutputParser):
        """输出转化为数组"""
        def parse(self, text: str):
            """Parse the output of an LLM call."""

            print(text)
            text = text.replace('*','')
            text = text.replace('\n',',')
            tmp = text.strip().split(",")
            ret = [i for i in tmp if i.strip()!='' and check2(i,MGC)]
            return ret
    list_flag ='错误'
    max_try_1 =3
    while '错误' in list_flag and max_try_1>0:
        if xqms.strip()!='':
            zongj_str = zongjiechain.invoke({'xqmc':xuqiumingcheng,'xqms':xqms}).content
        else:
            zongj_str = xqmc
        print(zongj_str)
        my_list = chanin.invoke({'xqmc':xuqiumingcheng,'n':qiefengeshu+5,'xqms':zongj_str}).content
        with open('E:\\输出.txt','a') as f:
            f.write('******************************')
            f.write(f'备选项:{my_list}\n')
        out_list = CommaSeparatedListOutputParser().parse(my_list)
        list_size = len(out_list)
        if list_size>=qiefengeshu:
            list_flag ='正确'
        time.sleep(10)
        max_try_1-=1
    zgc_str = ''
    now_cnt = 0
    print('@@@@@@@@@@@@@@@@@',out_list)
    for i in out_list:
        if now_cnt>=qiefengeshu:
            break
        now_cnt +=1
        print('-------------------------------------------------')
        print(i)
        hy = '错误'
        max_try = int(try_max)
        while '错误'  in hy and max_try>0:
            a= simp_sq_chain2.run({'xuqiu_name':f"{profex}-{i}",'his_zgc':zgc_str})
            hy = jcy_chain.invoke({'content':a})
            if '错误' not in hy:
                hy = check_list(a,MGC)
            max_try-=1
        zgc_str += gent_zgc(a)
        print('*****************',zgc_str)
        with open('E:\\输出.txt','a') as f:
            f.write('-------------------------------------------------\n')
            f.write(i)
            f.write('\n')
            f.write(a)
            f.write('\n')
            f.write('-------------------------------------------------\n')
        print(a)
        print('-------------------------------------------------')
    return  zgc_str
        # 获取simp_sq_chain 的最终输出


In [13]:
xqmc = """关于集团成员预警增设预警标签及集团预警任务派单优化的需求"""
xqms ="""
宽带渗透率,千兆渗透率,电视渗透率,家庭V网渗透率,视频彩铃渗透率,移动云盘渗透率,咪咕视频渗透率,地市组织ID,地市组织描述,区县组织ID,区县组织描述,业务类型,合同金额,到期日期,真库成员数,真库异网成员数,客户经理编码 （boss）,客户经理名称 （boss）,客户经理编码 （4a工号）,351开头的客户编码,客户经理地市代码,客户经理地市,客户经理区县代码,客户经理区县,所属重客地市代码,所属重客地市,所属重客区县代码,所属重客区县,企业员工数（核定员工数）,本网员工数（通讯录）,本网员工数（V网）,本网通讯录平均ARPU（元/月/人）,本网V网平均ARPU（元/月/人）,我网通讯录C客户份额（V网需要手动填写）,我司信息化收入（万元）,注册地址(V网),V网用户数（不限年龄）,V网用户数（18-60岁）,年累计出账收入(V网),主套平均折后收入(V网),近三月平均ARPU>50用户数(V网),近三月平均ARPU<=50用户数(V网),双卡用户数(V网),双卡中我网主卡数(V网),套餐共享主卡客户数(V网),宽带渗透率(V网),注册地址(通讯录),通讯录导入号码数(通讯录),导入验真号码数(通讯录),验真通过率(验真/导入)(通讯录),通讯录导入率(验真号码占比员工总数)(通讯录),通讯录成员规模保有率,通讯录成员价值保有率,拍照成员规模保有率,拍照成员价值保有率,移动用户数(通讯录),移动用户份额(通讯录),山西移动用户数(通讯录),山西移动客户份额(通讯录),LT号码数(通讯录),LT份额(通讯录),DX号码数(通讯录),DX份额(通讯录),GD号码数(通讯录),GD份额(通讯录),年累计出账收入(通讯录),主套平均价格(通讯录),近三月平均ARPU>50用户数(通讯录),近三月平均ARPU<=50用户数(通讯录),双卡用户数(通讯录),双卡中我网主卡数(通讯录),套餐共享主卡客户数(通讯录),V网客户数(通讯录),用户近三个月平均ARPU(V网),主套平均折后价格(通讯录),用户近三个月平均ARPU(通讯录),宽带渗透率(通讯录)
"""
profex = xqmc
n = 10
zgc_str = go(xqmc,xqms,n,profex)

在当前快速变化的电信市场环境中，为提升服务质量和客户满意度，我们作为行业领导者，需进一步精细化管理，特别是针对集团客户的服务与分析。为此，我们提出“集团成员预警增设预警标签及集团预警任务派单优化”这一关键需求，旨在通过数据洞察和流程优化，实现更为精准有效的集团客户维护与拓展。

需求背景中，我们关注到集团客户业务的复杂性和多样性，涉及从宽带、千兆接入到增值服务如电视、视频彩铃、移动云存储等多个层面的渗透率，以及客户基础信息、业务合同详情、客户经理管理效率等多维度数据。这些指标不仅反映了集团客户的使用现状，也预示了未来的市场趋势与潜在风险。随着市场竞争加剧，及时识别并响应集团客户的异动，对于保持业务稳定增长至关重要。

核心需求包括两大部分：一是预警标签的增设，二是预警任务派单流程的优化。预警标签的增设，意在对如宽带渗透率下降、重要服务使用率异常、合同即将到期等关键节点进行即时标注，使决策者能一目了然地捕捉到集团客户可能面临的挑战或机遇。通过引入诸如“潜在流失风险”、“服务升级机会”、“合同续约预警”等标签，不仅提升了数据的可读性，也为快速响应机制的建立奠定了基础。

派单优化则着重于提升处理效率和准确性。当某一预警被触发时，系统应自动识别相关客户经理，基于其负责的地市、区县乃至特定重客区域，将预警任务精准派发。同时，考虑客户经理的业务负荷和专业匹配度，优化派单算法，确保任务得到及时且专业的处理。此外，系统应支持反馈循环，记录处理结果，为后续的预警策略调整提供依据，形成闭环管理。

在数据层面，我们要求全面覆盖从宏观的业务类型、合同金额到微观的个人ARPU值、V网与通讯录的详细分析，甚至包括地域分布、员工结构及特定年龄段的用户行为分析，这些详尽的报表指标构成了集团客户全方位画像的基础。通过这些数据，我们能够深入理解集团客户的内部结构与外在需求，进而制定更为个性化的服务方案，增强客户粘性，提高市场份额。

尤其值得注意的是，对于通讯录和V网的深入分析，揭示了企业内部沟通习惯与员工使用偏好，这为我们提供了宝贵的线索，以开发更加符合现代企业需求的通信解决方案。通过分析V网和通讯录的用户规模、价值保有率及双卡使用情况等，我们能够精准定位服务改进点，进一步挖掘潜在收入增长点。

综上所述，此需求的实施不仅是技术升级的体现，更是服务理念的深化。它要求我们在大数据的海洋中精准捕捞，